## create gmail auth_config

In [13]:
import requests

# Your Composio API key
API_KEY = "ak_GepAqzYQ6d6WggBF_zot"   # 🔑 Replace with your actual API key

# API endpoint
url = "https://backend.composio.dev/api/v3/auth_configs"

# Request headers
headers = {
    "x-api-key": API_KEY,
    "Content-Type": "application/json"
}

# Request body
# Change "gmail" to the service/toolkit you want (example: "gmail", "slack", etc.)
payload = {
    "toolkit": {
        "slug": "gmail"
    }
}

# Send POST request
response = requests.post(url, headers=headers, json=payload)

# Handle the response
if response.ok:
    data = response.json()
    print("✅ Auth Config created successfully!")
    print("Response data:", data)
else:
    print("❌ Failed to create Auth Config")
    print("Status Code:", response.status_code)
    print("Response:", response.text)


✅ Auth Config created successfully!
Response data: {'toolkit': {'slug': 'gmail'}, 'auth_config': {'id': 'ac_hDe0DOfQlOXz', 'auth_scheme': 'OAUTH2', 'is_composio_managed': True, 'restrict_to_following_tools': []}}


## list of auth config items 

In [7]:
import requests

def list_auth_configs(api_key: str):
    """
    List all authentication configurations from Composio API.

    :param api_key: Your Composio API key
    :return: JSON list of auth configs or None if request fails
    """
    url = "https://backend.composio.dev/api/v3/auth_configs"
    headers = {
        "x-api-key": api_key
    }

    try:
        response = requests.get(url, headers=headers)

        if response.ok:
            print("✅ Auth configs fetched successfully.")
            return response.json()
        else:
            print("❌ Failed to fetch auth configs")
            print("Status Code:", response.status_code)
            print("Response:", response.text)
            return None

    except Exception as e:
        print("⚠️ Error:", str(e))
        return None


# 🧪 Example usage
if __name__ == "__main__":
    YOUR_API_KEY = "ak_GepAqzYQ6d6WggBF_zot"  # <-- Replace with your API key
    configs = list_auth_configs(YOUR_API_KEY)
    print("Auth Configs:", configs)


✅ Auth configs fetched successfully.
Auth Configs: {'items': [{'id': 'ac_busLmBvBTxkY', 'uuid': '922816ef-cb30-4293-bfbd-4117f1a4f633', 'name': 'auth_config_gmail_1757662096892', 'auth_scheme': 'OAUTH2', 'is_composio_managed': True, 'credentials': {'client_id': '****', 'client_secret': '****', 'scopes': [], 'oauth_redirect_uri': 'https://backend.composio.dev/api/v1/auth-apps/add'}, 'created_by': 'fb674baf-882a-4460-8bb6-366181f0c34a', 'created_at': '2025-09-12T07:28:16.894Z', 'last_updated_at': '2025-09-12T07:28:16.894Z', 'status': 'ENABLED', 'toolkit': {'slug': 'gmail', 'logo': 'https://cdn.jsdelivr.net/gh/ComposioHQ/open-logos@master/gmail.svg'}, 'no_of_connections': 0, 'tool_access_config': {'tools_for_connected_account_creation': [], 'tools_available_for_execution': []}, 'type': 'default', 'deprecated_params': {'default_connector_id': 'test-gmail-connector', 'member_uuid': 'fb674baf-882a-4460-8bb6-366181f0c34a', 'toolkit_id': 'a90e7d79-4f7a-4ff2-bd7d-19c78640b8f8', 'expected_input_

## get details 

In [8]:
import requests

def get_auth_config(api_key: str, nanoid: str):
    """
    Get a single authentication configuration by its nanoid.

    :param api_key: Your Composio API key
    :param nanoid: The unique ID of the auth config
    :return: JSON response with auth config details or None if failed
    """
    url = f"https://backend.composio.dev/api/v3/auth_configs/{nanoid}"
    headers = {
        "x-api-key": api_key
    }

    try:
        response = requests.get(url, headers=headers)

        if response.ok:
            print("✅ Auth config fetched successfully.")
            return response.json()
        else:
            print("❌ Failed to fetch auth config")
            print("Status Code:", response.status_code)
            print("Response:", response.text)
            return None

    except Exception as e:
        print("⚠️ Error:", str(e))
        return None


# 🧪 Example usage
if __name__ == "__main__":
    YOUR_API_KEY = "ak_GepAqzYQ6d6WggBF_zot"  # <-- Replace with your API key
    AUTH_CONFIG_ID = "ac_busLmBvBTxkY"   # <-- Replace this
    data = get_auth_config(YOUR_API_KEY, AUTH_CONFIG_ID)
    print("Auth Config Details:", data)


✅ Auth config fetched successfully.
Auth Config Details: {'id': 'ac_busLmBvBTxkY', 'uuid': '922816ef-cb30-4293-bfbd-4117f1a4f633', 'name': 'auth_config_gmail_1757662096892', 'auth_scheme': 'OAUTH2', 'is_composio_managed': True, 'credentials': {'client_id': '****', 'client_secret': '****', 'oauth_redirect_uri': 'https://backend.composio.dev/api/v1/auth-apps/add'}, 'created_by': 'member', 'created_at': '2025-09-12T07:28:16.894Z', 'status': 'ENABLED', 'toolkit': {'slug': 'gmail', 'logo': 'https://cdn.jsdelivr.net/gh/ComposioHQ/open-logos@master/gmail.svg'}, 'last_updated_at': '2025-09-12T07:28:16.894Z', 'no_of_connections': 0, 'expected_input_fields': [], 'restrict_to_following_tools': [], 'type': 'default', 'tool_access_config': {'tools_for_connected_account_creation': [], 'tools_available_for_execution': []}, 'deprecated_params': {'default_connector_id': 'test-gmail-connector', 'member_uuid': 'fb674baf-882a-4460-8bb6-366181f0c34a', 'toolkit_id': 'a90e7d79-4f7a-4ff2-bd7d-19c78640b8f8', '

## send message to user


In [10]:
import os
import requests
# from dotenv import load_dotenv

# load_dotenv()

# COMPOSIO_API_KEY = os.getenv("COMPOSIO_API_KEY")
# GMAIL_AUTH_CONFIG_ID = os.getenv("GMAIL_AUTH_CONFIG_ID")

def send_message(
    connected_account_id: str,
    to_email: str,
    subject: str,
    body: str
) -> dict:
    """
    Send an email via Gmail using Composio API.
    
    Args:
        connected_account_id (str): User's connected Gmail account ID
        to_email (str): Recipient email address
        subject (str): Email subject
        body (str): Email body content

    Returns:
        dict: API response
    """
    url = "https://backend.composio.dev/api/v3/actions/run"
    headers = {"x-api-key": COMPOSIO_API_KEY}
    payload = {
        "connectedAccountId": connected_account_id,
        "action": "gmail.sendEmail",
        "input": {
            "to": to_email,
            "subject": subject,
            "body": body,
        },
    }

    try:
        res = requests.post(url, headers=headers, json=payload)
        res.raise_for_status()
        return {"success": True, "response": res.json()}
    except requests.RequestException as e:
        return {"success": False, "error": str(e)}


## Delete auth config

In [14]:
import requests

def delete_auth_config(nanoid: str, api_key: str):
    """
    Delete an authentication configuration from Composio API with detailed debug.

    :param nanoid: The ID of the auth config to delete
    :param api_key: Your Composio API key
    :return: JSON response from API or None if request fails
    """
    url = f"https://backend.composio.dev/api/v3/auth_configs/{nanoid}"
    headers = {
        "x-api-key": api_key
    }

    try:
        print(f"🔹 Sending DELETE request to: {url}")
        response = requests.delete(url, headers=headers)

        print(f"🔹 Status Code: {response.status_code}")
        print(f"🔹 Response Headers: {response.headers}")
        print(f"🔹 Response Text: {response.text}")

        if response.ok:
            print(f"✅ Auth config {nanoid} deleted successfully.")
            return response.json()
        else:
            print(f"❌ Failed to delete auth config {nanoid}")
            # Extra debug info
            if response.status_code == 401:
                print("⚠️ Unauthorized: Check your API key.")
            elif response.status_code == 403:
                print("⚠️ Forbidden: API key does not have delete permissions.")
            elif response.status_code == 404:
                print("⚠️ Not Found: Check if the nanoid is correct.")
            return None

    except requests.exceptions.RequestException as e:
        print("⚠️ Request Exception:", str(e))
        return None


# 🧪 Example usage
if __name__ == "__main__":
    YOUR_API_KEY = "ak_GepAqzYQ6d6WggBF_zot"  # <-- Replace with your API key
    NANOID_TO_DELETE = "ac_my-IRuBzrA-G"             # <-- Replace with the config ID
    result = delete_auth_config(NANOID_TO_DELETE, YOUR_API_KEY)
    print("Delete Result:", result)


🔹 Sending DELETE request to: https://backend.composio.dev/api/v3/auth_configs/ac_my-IRuBzrA-G
🔹 Status Code: 200
🔹 Response Headers: {'Date': 'Fri, 12 Sep 2025 10:32:36 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Server': 'cloudflare', 'CF-Ray': '97debde3ec1646b9-BOM', 'CF-Cache-Status': 'DYNAMIC', 'Cache-Control': 'public, max-age=0, must-revalidate', 'Content-Encoding': 'gzip', 'Strict-Transport-Security': 'max-age=63072000', 'Vary': 'Origin, accept-encoding', 'access-control-allow-credentials': 'true', 'Cf-Placement': 'local-BOM', 'x-matched-path': '/api/v3/auth_configs/[nanoid]', 'x-powered-by': 'composio tech team', 'x-request-id': 'dba4b407-3f51-426f-bd77-b389fee2ba76', 'x-vercel-cache': 'MISS', 'x-vercel-id': 'bom1::iad1::r4zzl-1757673155949-e80494f75224'}
🔹 Response Text: {"success":true,"message":"Successfully deleted auth config"}
✅ Auth config ac_my-IRuBzrA-G deleted successfully.
Delete Result: {'success': True, 'me

## update config

In [ ]:
import requests

def update_auth_config(nanoid: str, api_key: str, update_data: dict):
    """
    Update an authentication configuration on Composio API.

    :param nanoid: The ID of the auth config to update
    :param api_key: Your Composio API key
    :param update_data: Dictionary containing fields to update
    :return: JSON response from API or None if request fails
    """
    url = f"https://backend.composio.dev/api/v3/auth_configs/{nanoid}"
    headers = {
        "x-api-key": api_key,
        "Content-Type": "application/json"
    }

    try:
        print(f"🔹 Sending PATCH request to: {url}")
        print(f"🔹 Update Data: {update_data}")

        response = requests.patch(url, headers=headers, json=update_data)

        print(f"🔹 Status Code: {response.status_code}")
        print(f"🔹 Response Text: {response.text}")

        if response.ok:
            print(f"✅ Auth config {nanoid} updated successfully.")
            return response.json()
        else:
            print(f"❌ Failed to update auth config {nanoid}")
            if response.status_code == 400:
                print("⚠️ Bad Request: Check your update data.")
            elif response.status_code == 404:
                print("⚠️ Not Found: Check if the nanoid is correct.")
            elif response.status_code == 401:
                print("⚠️ Unauthorized: Check your API key.")
            return None

    except requests.exceptions.RequestException as e:
        print("⚠️ Request Exception:", str(e))
        return None


# 🧪 Example usage
if __name__ == "__main__":
    YOUR_API_KEY = "ak_GepAqzYQ6d6WggBF_zot"  # <-- Replace with your API key
    NANOID_TO_UPDATE = "abc123xyz"             # <-- Replace with the config ID
    UPDATE_DATA = {"type": "custom"}           # <-- Fields to update

    result = update_auth_config(NANOID_TO_UPDATE, YOUR_API_KEY, UPDATE_DATA)
    print("Update Result:", result)


## Status 

In [17]:
import requests

def set_auth_config_status(nanoid: str, status: str, api_key: str):
    """
    Enable or disable an authentication configuration on Composio API.

    :param nanoid: The ID of the auth config
    :param status: "ENABLED" or "DISABLED"
    :param api_key: Your Composio API key
    :return: JSON response from API or None if request fails
    """
    url = f"https://backend.composio.dev/api/v3/auth_configs/{nanoid}/{status.upper()}"
    headers = {
        "x-api-key": api_key
    }

    try:
        print(f"🔹 Sending PATCH request to: {url}")

        response = requests.patch(url, headers=headers)

        print(f"🔹 Status Code: {response.status_code}")
        print(f"🔹 Response Text: {response.text}")

        if response.ok:
            print(f"✅ Auth config {nanoid} set to {status.upper()} successfully.")
            return response.json()
        else:
            print(f"❌ Failed to set auth config {nanoid} status to {status.upper()}")
            if response.status_code == 400:
                print("⚠️ Bad Request: Check if status is ENABLED or DISABLED.")
            elif response.status_code == 404:
                print("⚠️ Not Found: Check if the nanoid is correct.")
            elif response.status_code == 401:
                print("⚠️ Unauthorized: Check your API key.")
            return None

    except requests.exceptions.RequestException as e:
        print("⚠️ Request Exception:", str(e))
        return None


# 🧪 Example usage
if __name__ == "__main__":
    YOUR_API_KEY = "ak_GepAqzYQ6d6WggBF_zot"  # <-- Replace with your API key
    NANOID = "ac_hDe0DOfQlOXz"                       # <-- Replace with the config ID
    STATUS = "ENABLED"                         # or "DISABLED"

    result = set_auth_config_status(NANOID, STATUS, YOUR_API_KEY)
    print("Result:", result)


🔹 Sending PATCH request to: https://backend.composio.dev/api/v3/auth_configs/ac_hDe0DOfQlOXz/ENABLED
🔹 Status Code: 200
🔹 Response Text: {"success":true,"message":"Successfully updated auth config"}
✅ Auth config ac_hDe0DOfQlOXz set to ENABLED successfully.
Result: {'success': True, 'message': 'Successfully updated auth config'}


##  create_notion_oauth2_config

In [ ]:
import requests
from typing import Dict, Any, Optional

COMPOSIO_BASE_URL = "https://backend.composio.dev/api"  # adjust if needed


def create_notion_oauth2_config(
    api_key: str,
    client_id: str,
    client_secret: str,
    config_name: str = "Notion OAuth2 Config"
) -> Optional[Dict[str, Any]]:
    """
    Create a Notion OAuth2 authentication configuration on Composio API.

    :param api_key: Composio API key
    :param client_id: Notion OAuth2 client ID
    :param client_secret: Notion OAuth2 client secret
    :param config_name: Name for the auth config
    :return: JSON response from API if successful, else None
    """
    url = f"{COMPOSIO_BASE_URL}/v1/auth-configs"
    headers = {
        "x-api-key": api_key,
        "Content-Type": "application/json"
    }

    data = {
        "toolkit": "notion",
        "auth_type": "oauth2",
        "name": config_name,
        "config": {
            "client_id": client_id,
            "client_secret": client_secret
        }
    }

    try:
        print(f"🔹 Sending POST request to: {url}")
        print(f"🔹 Payload: {data}")

        response = requests.post(url, json=data, headers=headers)

        print(f"🔹 Status Code: {response.status_code}")
        print(f"🔹 Response Text: {response.text}")

        if response.ok:
            print("✅ Notion OAuth2 config created successfully.")
            return response.json()
        else:
            print("❌ Failed to create Notion OAuth2 config.")
            if response.status_code == 400:
                print("⚠️ Bad Request: Check your payload.")
            elif response.status_code == 401:
                print("⚠️ Unauthorized: Check your API key.")
            elif response.status_code == 409:
                print("⚠️ Conflict: Config with this name may already exist.")
            return None

    except requests.exceptions.RequestException as e:
        print("⚠️ Request Exception:", str(e))
        return None


# 🧪 Example usage
if __name__ == "__main__":
    YOUR_API_KEY = "ak_xxx"  # <-- Replace with your Composio API key
    CLIENT_ID = "your-notion-client-id"
    CLIENT_SECRET = "your-notion-client-secret"

    result = create_notion_oauth2_config(
        api_key=YOUR_API_KEY,
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET
    )

    print("Result:", result)


## create_gmail_bearer_token_config

In [ ]:
import requests
from typing import Dict, Any, Optional

COMPOSIO_BASE_URL = "https://backend.composio.dev/api"  # adjust if needed


def create_gmail_bearer_token_config(
    api_key: str,
    bearer_token: str,
    config_name: str = "Gmail Bearer Token Config"
) -> Optional[Dict[str, Any]]:
    """
    Create a Gmail Bearer Token authentication configuration on Composio API.

    :param api_key: Composio API key
    :param bearer_token: Gmail Bearer Token
    :param config_name: Name for the auth config
    :return: JSON response from API if successful, else None
    """
    url = f"{COMPOSIO_BASE_URL}/v1/auth-configs"
    headers = {
        "x-api-key": api_key,
        "Content-Type": "application/json"
    }

    data = {
        "toolkit": "gmail",
        "auth_type": "bearer_token",
        "name": config_name,
        "config": {
            "bearer_token": bearer_token
        }
    }

    try:
        print(f"🔹 Sending POST request to: {url}")
        print(f"🔹 Payload: {data}")

        response = requests.post(url, json=data, headers=headers)

        print(f"🔹 Status Code: {response.status_code}")
        print(f"🔹 Response Text: {response.text}")

        if response.ok:
            print("✅ Gmail Bearer Token config created successfully.")
            return response.json()
        else:
            print("❌ Failed to create Gmail Bearer Token config.")
            if response.status_code == 400:
                print("⚠️ Bad Request: Check your payload.")
            elif response.status_code == 401:
                print("⚠️ Unauthorized: Check your API key.")
            elif response.status_code == 409:
                print("⚠️ Conflict: Config with this name may already exist.")
            return None

    except requests.exceptions.RequestException as e:
        print("⚠️ Request Exception:", str(e))
        return None


# 🧪 Example usage
if __name__ == "__main__":
    YOUR_API_KEY = "ak_xxx"  # <-- Replace with your Composio API key
    BEARER_TOKEN = "your-gmail-bearer-token"

    result = create_gmail_bearer_token_config(
        api_key=YOUR_API_KEY,
        bearer_token=BEARER_TOKEN
    )

    print("Result:", result)


##  create_github_oauth2_config

In [ ]:
import requests
from typing import Dict, Any, Optional

COMPOSIO_BASE_URL = "https://backend.composio.dev/api"  # adjust if needed


def create_github_oauth2_config(
    api_key: str,
    client_id: str,
    client_secret: str,
    scopes: Optional[list] = None,
    config_name: str = "GitHub OAuth2 Config"
) -> Optional[Dict[str, Any]]:
    """
    Create a GitHub OAuth2 authentication configuration on Composio API.

    :param api_key: Composio API key
    :param client_id: GitHub OAuth2 client ID
    :param client_secret: GitHub OAuth2 client secret
    :param scopes: List of GitHub API scopes (optional)
    :param config_name: Name for the auth config
    :return: JSON response from API if successful, else None
    """
    if not scopes:
        scopes = ["repo", "user", "read:org"]

    url = f"{COMPOSIO_BASE_URL}/v1/auth-configs"
    headers = {
        "x-api-key": api_key,
        "Content-Type": "application/json"
    }

    data = {
        "toolkit": "github",
        "auth_type": "oauth2",
        "name": config_name,
        "config": {
            "client_id": client_id,
            "client_secret": client_secret,
            "scopes": scopes
        }
    }

    try:
        print(f"🔹 Sending POST request to: {url}")
        print(f"🔹 Payload: {data}")

        response = requests.post(url, json=data, headers=headers)

        print(f"🔹 Status Code: {response.status_code}")
        print(f"🔹 Response Text: {response.text}")

        if response.ok:
            print("✅ GitHub OAuth2 config created successfully.")
            return response.json()
        else:
            print("❌ Failed to create GitHub OAuth2 config.")
            if response.status_code == 400:
                print("⚠️ Bad Request: Check your payload.")
            elif response.status_code == 401:
                print("⚠️ Unauthorized: Check your API key.")
            elif response.status_code == 409:
                print("⚠️ Conflict: Config with this name may already exist.")
            return None

    except requests.exceptions.RequestException as e:
        print("⚠️ Request Exception:", str(e))
        return None


# 🧪 Example usage
if __name__ == "__main__":
    YOUR_API_KEY = "ak_xxx"  # <-- Replace with your Composio API key
    CLIENT_ID = "your-github-client-id"
    CLIENT_SECRET = "your-github-client-secret"

    result = create_github_oauth2_config(
        api_key=YOUR_API_KEY,
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET
    )

    print("Result:", result)


## create_notion_oauth2_config

In [ ]:
import requests
from typing import Dict, Any, Optional

COMPOSIO_BASE_URL = "https://backend.composio.dev/api"  # adjust if needed


def create_notion_oauth2_config(
    api_key: str,
    client_id: str,
    client_secret: str,
    config_name: str = "Notion OAuth2 Config"
) -> Optional[Dict[str, Any]]:
    """
    Create a Notion OAuth2 authentication configuration on Composio API.

    :param api_key: Composio API key
    :param client_id: Notion OAuth2 client ID
    :param client_secret: Notion OAuth2 client secret
    :param config_name: Name for the auth config
    :return: JSON response from API if successful, else None
    """
    url = f"{COMPOSIO_BASE_URL}/v1/auth-configs"
    headers = {
        "x-api-key": api_key,
        "Content-Type": "application/json"
    }

    data = {
        "toolkit": "notion",
        "auth_type": "oauth2",
        "name": config_name,
        "config": {
            "client_id": client_id,
            "client_secret": client_secret
        }
    }

    try:
        print(f"🔹 Sending POST request to: {url}")
        print(f"🔹 Payload: {data}")

        response = requests.post(url, json=data, headers=headers)

        print(f"🔹 Status Code: {response.status_code}")
        print(f"🔹 Response Text: {response.text}")

        if response.ok:
            print("✅ Notion OAuth2 config created successfully.")
            return response.json()
        else:
            print("❌ Failed to create Notion OAuth2 config.")
            if response.status_code == 400:
                print("⚠️ Bad Request: Check your payload.")
            elif response.status_code == 401:
                print("⚠️ Unauthorized: Check your API key.")
            elif response.status_code == 409:
                print("⚠️ Conflict: Config with this name may already exist.")
            return None

    except requests.exceptions.RequestException as e:
        print("⚠️ Request Exception:", str(e))
        return None


# 🧪 Example usage
if __name__ == "__main__":
    YOUR_API_KEY = "ak_xxx"  # <-- Replace with your Composio API key
    CLIENT_ID = "your-notion-client-id"
    CLIENT_SECRET = "your-notion-client-secret"

    result = create_notion_oauth2_config(
        api_key=YOUR_API_KEY,
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET
    )

    print("Result:", result)


## create_notion_api_key_config

In [ ]:
import requests
from typing import Dict, Any, Optional

COMPOSIO_BASE_URL = "https://backend.composio.dev/api"  # adjust if needed


def create_notion_api_key_config(
    api_key: str,
    notion_api_key: str,
    config_name: str = "Notion API Key Config"
) -> Optional[Dict[str, Any]]:
    """
    Create a Notion API Key authentication configuration on Composio API.

    :param api_key: Composio API key
    :param notion_api_key: Notion API key
    :param config_name: Name for the auth config
    :return: JSON response from API if successful, else None
    """
    url = f"{COMPOSIO_BASE_URL}/v1/auth-configs"
    headers = {
        "x-api-key": api_key,
        "Content-Type": "application/json"
    }

    data = {
        "toolkit": "notion",
        "auth_type": "api_key",
        "name": config_name,
        "config": {
            "api_key": notion_api_key
        }
    }

    try:
        print(f"🔹 Sending POST request to: {url}")
        print(f"🔹 Payload: {data}")

        response = requests.post(url, json=data, headers=headers)

        print(f"🔹 Status Code: {response.status_code}")
        print(f"🔹 Response Text: {response.text}")

        if response.ok:
            print("✅ Notion API Key config created successfully.")
            return response.json()
        else:
            print("❌ Failed to create Notion API Key config.")
            if response.status_code == 400:
                print("⚠️ Bad Request: Check your payload.")
            elif response.status_code == 401:
                print("⚠️ Unauthorized: Check your API key.")
            elif response.status_code == 409:
                print("⚠️ Conflict: Config with this name may already exist.")
            return None

    except requests.exceptions.RequestException as e:
        print("⚠️ Request Exception:", str(e))
        return None


# 🧪 Example usage
if __name__ == "__main__":
    YOUR_API_KEY = "ak_xxx"  # <-- Replace with your Composio API key
    NOTION_API_KEY = "secret_notion_api_key"

    result = create_notion_api_key_config(
        api_key=YOUR_API_KEY,
        notion_api_key=NOTION_API_KEY
    )

    print("Result:", result)


## create_slack_oauth2_config

In [ ]:
import requests
from typing import Dict, Any, Optional

COMPOSIO_BASE_URL = "https://backend.composio.dev/api"  # adjust if needed


def create_slack_oauth2_config(
    api_key: str,
    client_id: str,
    client_secret: str,
    scopes: Optional[list] = None,
    config_name: str = "Slack OAuth2 Config"
) -> Optional[Dict[str, Any]]:
    """
    Create a Slack OAuth2 authentication configuration on Composio API.

    :param api_key: Composio API key
    :param client_id: Slack OAuth2 client ID
    :param client_secret: Slack OAuth2 client secret
    :param scopes: List of Slack API scopes (optional)
    :param config_name: Name for the auth config
    :return: JSON response from API if successful, else None
    """
    if not scopes:
        scopes = ["channels:read", "chat:write", "users:read"]

    url = f"{COMPOSIO_BASE_URL}/v1/auth-configs"
    headers = {
        "x-api-key": api_key,
        "Content-Type": "application/json"
    }

    data = {
        "toolkit": "slack",
        "auth_type": "oauth2",
        "name": config_name,
        "config": {
            "client_id": client_id,
            "client_secret": client_secret,
            "scopes": scopes
        }
    }

    try:
        print(f"🔹 Sending POST request to: {url}")
        print(f"🔹 Payload: {data}")

        response = requests.post(url, json=data, headers=headers)

        print(f"🔹 Status Code: {response.status_code}")
        print(f"🔹 Response Text: {response.text}")

        if response.ok:
            print("✅ Slack OAuth2 config created successfully.")
            return response.json()
        else:
            print("❌ Failed to create Slack OAuth2 config.")
            if response.status_code == 400:
                print("⚠️ Bad Request: Check your payload.")
            elif response.status_code == 401:
                print("⚠️ Unauthorized: Check your API key.")
            elif response.status_code == 409:
                print("⚠️ Conflict: Config with this name may already exist.")
            return None

    except requests.exceptions.RequestException as e:
        print("⚠️ Request Exception:", str(e))
        return None


# 🧪 Example usage
if __name__ == "__main__":
    YOUR_API_KEY = "ak_xxx"  # <-- Replace with your Composio API key
    CLIENT_ID = "your-slack-client-id"
    CLIENT_SECRET = "your-slack-client-secret"

    result = create_slack_oauth2_config(
        api_key=YOUR_API_KEY,
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET
    )

    print("Result:", result)


## slack bearer token


In [ ]:

def create_slack_bearer_token_config(
    api_key: str,
    bot_token: str,
    config_name: str = "Slack Bearer Token Config"
) -> Dict[str, Any]:
    """
    Create Slack Bearer Token authentication configuration.
    """
    url = f"{COMPOSIO_BASE_URL}/v1/auth-configs"
    headers = {"x-api-key": api_key, "Content-Type": "application/json"}
    
    data = {
        "toolkit": "slack",
        "auth_type": "bearer_token",
        "name": config_name,
        "config": {
            "bot_token": bot_token
        }
    }
    
    response = requests.post(url, json=data, headers=headers)
    response.raise_for_status()
    return response.json()


In [ ]:

def create_google_calendar_oauth2_config(
    api_key: str,
    client_id: str,
    client_secret: str,
    scopes: Optional[list] = None,
    config_name: str = "Google Calendar OAuth2 Config"
) -> Dict[str, Any]:
    """
    Create Google Calendar OAuth2 authentication configuration.
    """
    if not scopes:
        scopes = [
            "https://www.googleapis.com/auth/calendar",
            "https://www.googleapis.com/auth/calendar.events"
        ]
    
    url = f"{COMPOSIO_BASE_URL}/v1/auth-configs"
    headers = {"x-api-key": api_key, "Content-Type": "application/json"}
    
    data = {
        "toolkit": "googlecalendar",
        "auth_type": "oauth2",
        "name": config_name,
        "config": {
            "client_id": client_id,
            "client_secret": client_secret,
            "scopes": scopes
        }
    }
    
    response = requests.post(url, json=data, headers=headers)
    response.raise_for_status()
    return response.json()


In [ ]:
def create_google_calendar_bearer_token_config(
    api_key: str,
    access_token: str,
    config_name: str = "Google Calendar Bearer Token Config"
) -> Dict[str, Any]:
    """
    Create Google Calendar Bearer Token authentication configuration.
    """
    url = f"{COMPOSIO_BASE_URL}/v1/auth-configs"
    headers = {"x-api-key": api_key, "Content-Type": "application/json"}
    
    data = {
        "toolkit": "googlecalendar",
        "auth_type": "bearer_token",
        "name": config_name,
        "config": {
            "access_token": access_token
        }
    }
    
    response = requests.post(url, json=data, headers=headers)
    response.raise_for_status()
    return response.json()

In [ ]:
def create_google_sheets_oauth2_config(
    api_key: str,
    client_id: str,
    client_secret: str,
    scopes: Optional[list] = None,
    config_name: str = "Google Sheets OAuth2 Config"
) -> Dict[str, Any]:
    """
    Create Google Sheets OAuth2 authentication configuration.
    """
    if not scopes:
        scopes = [
            "https://www.googleapis.com/auth/spreadsheets",
            "https://www.googleapis.com/auth/drive"
        ]
    
    url = f"{COMPOSIO_BASE_URL}/v1/auth-configs"
    headers = {"x-api-key": api_key, "Content-Type": "application/json"}
    
    data = {
        "toolkit": "googlesheets",
        "auth_type": "oauth2",
        "name": config_name,
        "config": {
            "client_id": client_id,
            "client_secret": client_secret,
            "scopes": scopes
        }
    }
    
    response = requests.post(url, json=data, headers=headers)
    response.raise_for_status()
    return response.json()

In [ ]:

def create_supabase_oauth2_config(
    api_key: str,
    supabase_url: str,
    supabase_anon_key: str,
    config_name: str = "Supabase OAuth2 Config"
) -> Dict[str, Any]:
    """
    Create Supabase OAuth2 authentication configuration.
    """
    url = f"{COMPOSIO_BASE_URL}/v1/auth-configs"
    headers = {"x-api-key": api_key, "Content-Type": "application/json"}
    
    data = {
        "toolkit": "supabase",
        "auth_type": "oauth2",
        "name": config_name,
        "config": {
            "supabase_url": supabase_url,
            "supabase_anon_key": supabase_anon_key
        }
    }
    
    response = requests.post(url, json=data, headers=headers)
    response.raise_for_status()
    return response.json()

In [ ]:

def create_supabase_api_key_config(
    api_key: str,
    supabase_url: str,
    supabase_service_role_key: str,
    config_name: str = "Supabase API Key Config"
) -> Dict[str, Any]:
    """
    Create Supabase API Key authentication configuration.
    """
    url = f"{COMPOSIO_BASE_URL}/v1/auth-configs"
    headers = {"x-api-key": api_key, "Content-Type": "application/json"}
    
    data = {
        "toolkit": "supabase",
        "auth_type": "api_key",
        "name": config_name,
        "config": {
            "supabase_url": supabase_url,
            "service_role_key": supabase_service_role_key
        }
    }
    
    response = requests.post(url, json=data, headers=headers)
    response.raise_for_status()
    return response.json()

In [ ]:

# ===============================
# OUTLOOK TOOLKIT FUNCTIONS
# ===============================

def create_outlook_oauth2_config(
    api_key: str,
    client_id: str,
    client_secret: str,
    tenant_id: str,
    scopes: Optional[list] = None,
    config_name: str = "Outlook OAuth2 Config"
) -> Dict[str, Any]:
    """
    Create Outlook OAuth2 authentication configuration.
    """
    if not scopes:
        scopes = [
            "https://graph.microsoft.com/Mail.Read",
            "https://graph.microsoft.com/Mail.Send"
        ]
    
    url = f"{COMPOSIO_BASE_URL}/v1/auth-configs"
    headers = {"x-api-key": api_key, "Content-Type": "application/json"}
    
    data = {
        "toolkit": "outlook",
        "auth_type": "oauth2",
        "name": config_name,
        "config": {
            "client_id": client_id,
            "client_secret": client_secret,
            "tenant_id": tenant_id,
            "scopes": scopes
        }
    }
    
    response = requests.post(url, json=data, headers=headers)
    response.raise_for_status()
    return response.json()

In [ ]:

# ===============================
# PERPLEXITY TOOLKIT FUNCTIONS
# ===============================

def create_perplexity_api_key_config(
    api_key: str,
    perplexity_api_key: str,
    config_name: str = "Perplexity API Key Config"
) -> Dict[str, Any]:
    """
    Create Perplexity API Key authentication configuration.
    """
    url = f"{COMPOSIO_BASE_URL}/v1/auth-configs"
    headers = {"x-api-key": api_key, "Content-Type": "application/json"}
    
    data = {
        "toolkit": "perplexity",
        "auth_type": "api_key",
        "name": config_name,
        "config": {
            "api_key": perplexity_api_key
        }
    }
    
    response = requests.post(url, json=data, headers=headers)
    response.raise_for_status()
    return response.json()


In [ ]:

# ===============================
# COMPOSIO TOOLKIT FUNCTIONS  
# ===============================

def create_composio_no_auth_config(
    api_key: str,
    config_name: str = "Composio No Auth Config"
) -> Dict[str, Any]:
    """
    Create Composio No Auth configuration (internal toolkit).
    """
    url = f"{COMPOSIO_BASE_URL}/v1/auth-configs"
    headers = {"x-api-key": api_key, "Content-Type": "application/json"}
    
    data = {
        "toolkit": "composio",
        "auth_type": "no_auth",
        "name": config_name,
        "config": {}
    }
    
    response = requests.post(url, json=data, headers=headers)
    response.raise_for_status()
    return response.json()

In [ ]:

# ===============================
# GENERIC TOOLKIT FUNCTIONS
# ===============================

def get_auth_config(api_key: str, nanoid: str) -> Dict[str, Any]:
    """
    Get authentication configuration by nanoid.
    """
    url = f"{COMPOSIO_BASE_URL}/v1/auth-configs/{nanoid}"
    headers = {"x-api-key": api_key}
    
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

def list_auth_configs(api_key: str, toolkit: Optional[str] = None) -> Dict[str, Any]:
    """
    List all authentication configurations.
    """
    url = f"{COMPOSIO_BASE_URL}/v1/auth-configs"
    headers = {"x-api-key": api_key}
    params = {"toolkit": toolkit} if toolkit else {}
    
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()

def delete_auth_config(api_key: str, nanoid: str) -> Dict[str, Any]:
    """
    Delete authentication configuration.
    """
    url = f"{COMPOSIO_BASE_URL}/v1/auth-configs/{nanoid}"
    headers = {"x-api-key": api_key}
    
    response = requests.delete(url, headers=headers)
    response.raise_for_status()
    return response.json()

def update_auth_config(
    api_key: str,
    nanoid: str,
    config_data: Dict[str, Any],
    name: Optional[str] = None
) -> Dict[str, Any]:
    """
    Update authentication configuration.
    """
    url = f"{COMPOSIO_BASE_URL}/v1/auth-configs/{nanoid}"
    headers = {"x-api-key": api_key, "Content-Type": "application/json"}
    
    data = {"config": config_data}
    if name:
        data["name"] = name
    
    response = requests.put(url, json=data, headers=headers)
    response.raise_for_status()
    return response.json()
